In [85]:
%load_ext autoreload
%autoreload 2
import sys
import os
project_root = os.path.abspath("/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis")
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
import numpy as np 
import utils
from utils import *
from pathlib import Path
import harp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import aeon.io.video as video
from ipywidgets import widgets
from IPython.display import display
import re
import os
import zoneinfo
from datetime import datetime, timezone
from functools import reduce
from src.processing import detect_settings
import src.processing.detect_stage as detect_stage_module

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
# Loading experiments --> just define the SUBJID and DATE
root = load_experiment(26, 20250903) #can add index for multiple experiments; index=0 as default
stage = detect_stage_module.detect_stage(root)

Using subject directory: /Volumes/harris/hypnose/rawdata/sub-026_id-077
Loaded experiment: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-117_date-20250903/behav/2025-09-03T14-12-09
Detected stage: QuintupleProbeA_Location0
Hidden rule index: 0


In [87]:
data = load_all_streams(root)

Loading data streams from: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-117_date-20250903/behav/2025-09-03T14-12-09
Loaded heartbeat data
Calculated real-time offset: 44411 days, 11:08:47
Created timestamp interpolation mapping
Loaded digital_input_data
Loaded output_set
Loaded output_clear
Loaded olfactometer_valves_0
Loaded olfactometer_valves_1
Loaded olfactometer_end_0
Loaded analog_data
Loaded flow_meter
Loaded video_data
Loaded pulse_supply_1
Loaded pulse_supply_2
Created odour_led

Applying time corrections to all data streams...
Applied correction to digital_input_data
Applied correction to output_set
Applied correction to output_clear
Applied correction to olfactometer_valves_0
Applied correction to olfactometer_valves_1
Applied correction to olfactometer_end_0
Applied correction to analog_data
Applied correction to flow_meter
Applied correction to video_data
Applied correction to pulse_supply_1
Applied correction to pulse_supply_2
Applied correction to odour_led

Data l

In [88]:
events = load_experiment_events(root)

Loading experiment events...
Loaded heartbeat data for timing synchronization
Calculated real-time offset: 44411 days, 11:08:47
Created timestamp interpolation mapping
Found 2 experiment event files
Processing event file: ExperimentEvents_1904-01-31T04-00-00.csv with 788 rows
Using Seconds column with interpolation
Applied real-time offset: 44411 days, 11:08:47
Found Value column with values: ['ChooseRandomSequence' 'SampleRewardCondition' 'InitiationSequence'
 'EndInitiation' 'Reset' 'AwaitReward']
Found 137 EndInitiation events
Found 138 InitiationSequence events
Found 137 Reset events
Found 100 AwaitReward events
Found 138 SampleRewardCondition events
Found 138 ChooseRandomSequence events
Processing event file: ExperimentEvents_1904-01-31T05-00-00.csv with 611 rows
Using Seconds column with interpolation
Applied real-time offset: 44411 days, 11:08:47
Found Value column with values: ['EndInitiation' 'AwaitReward' 'Reset' 'ChooseRandomSequence'
 'SampleRewardCondition' 'InitiationSequ

In [89]:
odor_map = load_odor_mapping(root, data)
print("Data streams loaded:", list(data.keys()))
print("Event types loaded:", list(events.keys()))
print("Odor mapping:", odor_map['odour_to_olfactometer_map'])

Loading odor mapping from session settings...
Loaded session settings
Found 10 valve configurations for olfactometer 0
Found 10 valve configurations for olfactometer 1
Created valve-to-command mapping: {'00': 3, '01': 7, '02': 5, '03': 6, '10': 4, '11': 8, '12': 9, '13': 2}
Created odor mapping: [['OdorA', 'OdorE', 'OdorC', 'OdorD'], ['OdorB', 'OdorF', 'OdorG', 'Purge']]
Data streams loaded: ['heartbeat', 'real_time_offset', 'timestamp_to_time', 'interpolate_time', 'digital_input_data', 'output_set', 'output_clear', 'olfactometer_valves_0', 'olfactometer_valves_1', 'olfactometer_end_0', 'analog_data', 'flow_meter', 'video_reader', 'video_data', 'pulse_supply_1', 'pulse_supply_2', 'odour_led', 'behavior_reader', 'olfactometer_reader']
Event types loaded: ['combined_initiation_sequence_df', 'combined_end_initiation_df', 'combined_await_reward_df', 'combined_reset_df', 'combined_choose_random_sequence_df', 'combined_sample_reward_condition_df']
Odor mapping: [['OdorA', 'OdorE', 'OdorC', '

In [98]:
trial_counts = detect_trials(data, events, root, verbose=True)

TRIAL DETECTION - METHOD 3: Simplified
Parameters: minimum_sampling_time=200.0ms, sample_offset_time=200.0ms

InitiationSequence 0: 2025-09-03 15:13:32.409984
  Found 7 poke periods
    Starting attempt 1 at 2025-09-03 15:13:33.146752
      Poke 1: 163.0ms (total: 163.0ms)
      Poke 2: gap 11.9ms + 203.4ms (total: 378.3ms)
      SUCCESS: 378.3ms continuous poke (≥200.0ms)

InitiationSequence 1: 2025-09-03 15:13:44.274976
  Found 2 poke periods
    Starting attempt 1 at 2025-09-03 15:13:59.238432
      Poke 1: 330.5ms (total: 330.5ms)
      SUCCESS: 330.5ms continuous poke (≥200.0ms)

InitiationSequence 2: 2025-09-03 15:14:04.980000
  Found 8 poke periods
    Starting attempt 1 at 2025-09-03 15:14:06.775360
      Poke 1: 207.5ms (total: 207.5ms)
      SUCCESS: 207.5ms continuous poke (≥200.0ms)

InitiationSequence 3: 2025-09-03 15:14:17.224992
  Found 16 poke periods
    Starting attempt 1 at 2025-09-03 15:14:20.436448
      Poke 1: 595.1ms (total: 595.1ms)
      SUCCESS: 595.1ms conti

In [102]:
trial_outcomes_complete = classify_and_analyze_with_response_times(data, events, trial_counts, odor_map, stage, root)

CLASSIFYING TRIAL OUTCOMES (NO RESPONSE-TIME) WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS
Sample offset time: 200.0 ms
Minimum sampling time: 200.0 ms
Response time window: 2.5 s
Hidden rule location extracted: Location0 (index 0, position 1)
Found 1031 total valve activations (excluding Purge)
Analyzing 240 initiated trials...
Found 186 AwaitReward events
Found 185 total supply port activities
Hidden Rule Odors inferred: ['OdorA', 'OdorB']

TRIAL CLASSIFICATION RESULTS WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS (NO RESPONSE-TIME):
Hidden Rule Location: Position 1 (index 0)

Total attempts: 274
-- Non-initiated sequences (total): 34 (12.4%)
    -- Position 1 attempts within trials 8 (23.5%)
    -- Baseline non-initiated sequences 26 (76.5%)
-- Initiated sequences (trials): 240 (87.6%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 186 (77.5%)
   -- Hidden Rule trials (HR): 15 (6.2%)
   -- Hidden Rule Missed (HR_missed): 1 (0.4%)
-- Aborted sequences: 54 (22.5%)
   -- Aborted

# Testing of new functions

# Debugging Functions:



In [70]:
def check_data_availability(data, events, odor_map):
    """Quick check of all data streams for availability and content"""
    
    print("=" * 60)
    print("DATA AVAILABILITY CHECK")
    print("=" * 60)
    
    # Check main data streams
    print("\n📊 DATA STREAMS:")
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({value.shape})")
            if has_data:
                print(f"    Columns: {value.columns.tolist()}")
                print(f"    Sample:\n{value.head(3).to_string(max_cols=5)}")
                print()
        elif isinstance(value, pd.Series):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({len(value)} items)")
            if has_data:
                print(f"    Sample: {value.head(3).tolist()}")
                print()
        else:
            print(f"  {key}: {type(value).__name__}")
    
    # Check event streams
    print("\n🎯 EVENT STREAMS:")
    for key, value in events.items():
        if isinstance(value, pd.DataFrame):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({value.shape})")
            if has_data:
                print(f"    Columns: {value.columns.tolist()}")
                print(f"    Sample:\n{value.head(3).to_string()}")
                print()
    
    # Check odor mapping
    print("\n🌸 ODOR MAPPING:")
    print(f"  odour_to_olfactometer_map: {odor_map['odour_to_olfactometer_map']}")
    print(f"  valve_to_odor: {odor_map['valve_to_odor']}")
    
    # Check digital input ports specifically
    print("\n🔌 DIGITAL INPUT PORTS:")
    if 'digital_input_data' in data and not data['digital_input_data'].empty:
        digital_data = data['digital_input_data']
        for col in digital_data.columns:
            port_data = digital_data[col]
            has_activity = port_data.sum() > 0  # Check if any True values
            print(f"  {col}: {'✅ ACTIVE' if has_activity else '❌ NO ACTIVITY'} ({port_data.sum()} events)")
            if has_activity:
                # Show first few ON events
                on_events = port_data[port_data == True].head(3)
                print(f"    First ON events: {on_events.index.tolist()}")
    
    # Check output ports (LED control)
    print("\n💡 OUTPUT PORTS:")
    for stream_name in ['output_set', 'output_clear']:
        if stream_name in data and not data[stream_name].empty:
            output_data = data[stream_name]
            print(f"  {stream_name}:")
            for col in output_data.columns:
                port_data = output_data[col]
                has_activity = port_data.sum() > 0
                print(f"    {col}: {'✅ ACTIVE' if has_activity else '❌ NO ACTIVITY'} ({port_data.sum()} events)")

# Run the check
check_data_availability(data, events, odor_map)

DATA AVAILABILITY CHECK

📊 DATA STREAMS:
  heartbeat: ✅ HAS DATA ((9961, 2))
    Columns: ['Time', 'TimestampSeconds']
    Sample:
                 Time  TimestampSeconds
0 1904-01-12 03:42:00            963720
1 1904-01-12 03:42:01            963721
2 1904-01-12 03:42:02            963722

  real_time_offset: timedelta
  timestamp_to_time: ✅ HAS DATA (9961 items)
    Sample: [Timestamp('1904-01-12 03:42:00'), Timestamp('1904-01-12 03:42:01'), Timestamp('1904-01-12 03:42:02')]

  interpolate_time: function
  digital_input_data: ✅ HAS DATA ((6870, 4))
    Columns: ['DIPort0', 'DIPort1', 'DIPort2', 'DI3']
    Sample:
                            DIPort0  DIPort1  DIPort2    DI3
Time                                                        
2025-08-15 14:52:39.792832    False    False     True  False
2025-08-15 14:52:39.861536    False    False    False  False
2025-08-15 14:52:53.775296    False    False     True  False

  output_set: ✅ HAS DATA ((805, 14))
    Columns: ['DOPort0', 'DOPort1'

In [ ]:
# Quick code to look for brief purge events in between odors in completed trials

import pandas as pd
import numpy as np
import re
from bisect import bisect_left, bisect_right

def list_short_purge_between_odors(trial_outcomes_complete, odor_map, threshold_ms=200.0, onset_slack_ms=50.0, verbose=True):
    """
    Find Purge events shorter than threshold_ms that occur between distinct odor presentations
    in completed trials. A Purge onset is counted if it lies within:
       [current_odor_end - onset_slack_ms, next_odor_start + onset_slack_ms]
    Returns a list of dicts (one per event) and prints a summary.
    """

    # --- Helpers to resolve Purge columns ---
    def _ci_get(d, key):
        if not isinstance(d, dict):
            return None
        lk = str(key).lower()
        for k, v in d.items():
            if str(k).lower() == lk:
                return v
        return None

    def _col_to_idx(col, row_len=None):
        if isinstance(col, (int, np.integer)):
            idx_num = int(col)
        else:
            m = re.search(r'(\d+)', str(col))
            if not m:
                return None
            idx_num = int(m.group(1))
        if row_len is None:
            return idx_num
        if 0 <= idx_num < row_len:
            return idx_num
        if 1 <= idx_num <= row_len:
            return idx_num - 1
        return None

    def _resolve_odor_name(odor_map, olf_id, idx, col=None):
        v2o = odor_map.get("valve_to_odor", {})
        if isinstance(v2o, dict):
            name = v2o.get((olf_id, idx))
            if name is None and col is not None:
                name = v2o.get(col)
            if name is None:
                name = v2o.get(idx)
            if isinstance(name, str):
                return name
        grid = odor_map.get("odour_to_olfactometer_map") or odor_map.get("odor_to_olfactometer_map")
        if isinstance(grid, (list, tuple)) and len(grid) > olf_id and isinstance(grid[olf_id], (list, tuple)):
            row = grid[olf_id]
            if 0 <= idx < len(row):
                return row[idx]
        return None

    def _purge_columns(odor_map):
        cols = []
        olf_valves = odor_map["olfactometer_valves"]
        grid = odor_map.get("odour_to_olfactometer_map") or odor_map.get("odor_to_olfactometer_map")
        for olf_id, df in olf_valves.items():
            if df is None or getattr(df, "empty", True):
                continue
            row_len = len(grid[olf_id]) if isinstance(grid, (list, tuple)) and len(grid) > olf_id else None
            for col in df.columns:
                idx = _col_to_idx(col, row_len=row_len)
                if idx is None:
                    continue
                name = _resolve_odor_name(odor_map, olf_id, idx, col=col)
                if isinstance(name, str) and name.lower() == "purge":
                    cols.append((olf_id, idx, col))
        return cols

    def _purge_events(odor_map):
        """Return list of dicts: {start,end,duration_ms,olf_id,idx,col} across all purge columns."""
        evs = []
        for olf_id, idx, col in _purge_columns(odor_map):
            df = odor_map["olfactometer_valves"][olf_id]
            if df is None or getattr(df, "empty", True):
                continue
            s = df[col].astype(bool)
            rises = s & ~s.shift(1, fill_value=False)
            falls = ~s & s.shift(1, fill_value=False)
            starts = list(s.index[rises])
            ends = list(s.index[falls])
            i = j = 0
            while i < len(starts) and j < len(ends):
                if ends[j] <= starts[i]:
                    j += 1
                    continue
                dur_ms = (ends[j] - starts[i]).total_seconds() * 1000.0
                evs.append({
                    "start": starts[i],
                    "end": ends[j],
                    "duration_ms": dur_ms,
                    "olf_id": olf_id,
                    "idx": idx,
                    "col": col,
                })
                i += 1
                j += 1
        evs.sort(key=lambda e: e["start"])
        return evs

    # --- Build completed trials dataframe and inter-odor windows ---
    cls = trial_outcomes_complete.get("classification", trial_outcomes_complete)
    completed_keys = [
        "completed_sequence_rewarded",
        "completed_sequence_unrewarded",
        "completed_sequence_reward_timeout",
    ]
    completed_dfs = [cls[k] for k in completed_keys if k in cls and isinstance(cls[k], pd.DataFrame)]
    completed_df = pd.concat(completed_dfs, ignore_index=True) if completed_dfs else pd.DataFrame()

    def _trial_id(row):
        for k in ["trial_id", "trial_index", "sequence_index", "Trial", "Sequence", "trial"]:
            if k in row and pd.notna(row[k]):
                return row[k]
        return row.name

    # Collect inter-odor windows per trial: [(trial_id, pos_i, pos_j, win_start, win_end)]
    windows = []
    for _, row in completed_df.iterrows():
        pov = row.get("position_valve_times")
        if not isinstance(pov, dict) or not pov:
            continue
        tid = _trial_id(row)
        # sort positions by position number
        positions = sorted([p for p in pov.keys() if isinstance(p, (int, np.integer))])
        # windows between consecutive positions
        for i in range(len(positions) - 1):
            p_i = positions[i]
            p_j = positions[i + 1]
            end_i = pov[p_i].get("valve_end")
            start_j = pov[p_j].get("valve_start")
            if pd.isna(end_i) or pd.isna(start_j) or end_i is None or start_j is None:
                continue
            if end_i >= start_j:
                # overlapping/invalid; skip
                continue
            win_start = end_i - pd.Timedelta(milliseconds=onset_slack_ms)
            win_end = start_j + pd.Timedelta(milliseconds=onset_slack_ms)
            windows.append((tid, p_i, p_j, win_start, win_end))

    # --- Scan purge events and match onsets within windows ---
    purge_events = _purge_events(odor_map)
    starts = [e["start"] for e in purge_events]

    matches = []
    for tid, p_i, p_j, ws, we in windows:
        lo = bisect_left(starts, ws)
        hi = bisect_right(starts, we)
        for k in range(lo, hi):
            e = purge_events[k]
            if e["duration_ms"] < threshold_ms:
                matches.append({
                    "trial_id": tid,
                    "from_pos": p_i,
                    "to_pos": p_j,
                    "start": e["start"],
                    "end": e["end"],
                    "duration_ms": e["duration_ms"],
                    "olf_id": e["olf_id"],
                    "col": e["col"],
                })

    # Summary/print
    if verbose:
        print(f"Short Purge events (< {threshold_ms} ms) between odors in completed trials "
              f"(onset slack ±{onset_slack_ms} ms): {len(matches)}")
        by_trial = {}
        for m in matches:
            by_trial.setdefault(m["trial_id"], 0)
            by_trial[m["trial_id"]] += 1
        if matches:
            for m in sorted(matches, key=lambda x: (x["trial_id"], x["start"])):
                print(f"- trial {m['trial_id']} pos {m['from_pos']}->{m['to_pos']}: "
                      f"{m['start'].isoformat()} -> {m['end'].isoformat()} "
                      f"({m['duration_ms']:.1f} ms)  olf {m['olf_id']} col '{m['col']}'")
            print("\nCounts by trial:")
            for tid in sorted(by_trial):
                print(f"  trial {tid}: {by_trial[tid]}")

    return matches

# Example usage:
matches = list_short_purge_between_odors(trial_outcomes_complete, odor_map, threshold_ms=200.0, onset_slack_ms=100.0, verbose=True)
len(matches)

Short Purge events (< 200.0 ms) between odors in completed trials (onset slack ±100.0 ms): 4
- trial 107 pos 1->2: 2025-09-03T15:58:14.353504 -> 2025-09-03T15:58:14.547488 (194.0 ms)  olf 1 col 'Valve3'
- trial 131 pos 2->3: 2025-09-03T16:06:56.078496 -> 2025-09-03T16:06:56.155488 (77.0 ms)  olf 1 col 'Valve3'
- trial 186 pos 4->5: 2025-09-03T16:31:11.001504 -> 2025-09-03T16:31:11.064479999 (63.0 ms)  olf 1 col 'Valve3'
- trial 190 pos 3->4: 2025-09-03T16:32:58.143488 -> 2025-09-03T16:32:58.208480 (65.0 ms)  olf 1 col 'Valve3'

Counts by trial:
  trial 107: 1
  trial 131: 1
  trial 186: 1
  trial 190: 1


4

In [ ]:
# Identify trial IDs for trials with poke time bewlow threshold 
threshold_ms = 200.0

# Get the classification DataFrames
cls = trial_outcomes_complete.get("classification", trial_outcomes_complete)

completed_keys = [
    "completed_sequence_rewarded",
    "completed_sequence_unrewarded",
    "completed_sequence_reward_timeout",
]
completed_dfs = [cls[k] for k in completed_keys if k in cls and isinstance(cls[k], pd.DataFrame)]
completed_df = pd.concat(completed_dfs, ignore_index=True) if completed_dfs else pd.DataFrame()

def _trial_id(row):
    for k in ["trial_id", "trial_index", "sequence_index", "Trial", "Sequence", "trial"]:
        if k in row and pd.notna(row[k]):
            return row[k]
    return row.name  # fallback

hits = []
for _, row in completed_df.iterrows():
    pos_pokes = row.get("position_poke_times")
    if not isinstance(pos_pokes, dict):
        continue
    tid = _trial_id(row)
    for pos, info in pos_pokes.items():
        if not isinstance(info, dict):
            continue
        ms = info.get("poke_time_ms")
        if ms is None:
            continue
        if ms < threshold_ms:
            odor = info.get("odor_name")
            hits.append((ms, tid, pos, odor))

# Print results sorted by poke time
if not hits:
    print(f"No completed-trial positions with poke_time_ms < {threshold_ms} ms found.")
else:
    hits.sort(key=lambda x: x[0])
    print(f"Trials with poke_time_ms < {threshold_ms} ms (n={len(hits)}):")
    for ms, tid, pos, odor in hits:
        print(f"- trial {tid}, position {pos}, {odor}: {ms:.1f} ms")

Trials with poke_time_ms < 200.0 ms (n=1):
- trial 4, position 2, OdorC: 138.1 ms


In [ ]:
# Can be used to test if Hidden Rule Odor Identities can be detected from the Schema file. 
session_settings, schema_settings = detect_settings.detect_settings(root)
print("HR index inferred:", schema_settings.get("hiddenRuleIndexInferred"))
print("HR odors inferred:", schema_settings.get("hiddenRuleOdorsInferred"))